## Example Message Generation Workflow

This notebook demonstrates the end-to-end workflow for creating, signing, and validating a PCL message.

1.  **Build**: Constructing a message using the `PCLMessageBuilder`.
2.  **Sign**: Creating a cryptographic signature to ensure message integrity and authenticity.
3.  **Verify**: Simulating the receiver's process of authenticating the message signature.
4.  **Validate**: Checking the message's structure (with JSON Schema) and semantic meaning (with SHACL).

In [13]:
import json
from jwcrypto import jwk
from pcl_exchange.builder import PCLMessageBuilder
from pcl_exchange.crypto import Signer, Verifier
from pcl_exchange.validation import validate_semantics, validate_structure

#### Establish Sender Identity
Before sending a message, the sender needs a cryptographic identity. We generate an `Ed25519` key pair, which is a modern and efficient elliptic curve signature algorithm. The public part of this key can be shared with receivers, while the private part is kept secret and used for signing.

The key's thumbprint serves as a unique identifier (`kid`) for the key.

In [14]:
# start by generating a sender key
sender_key = jwk.JWK.generate(kty='OKP', crv='Ed25519')
print(f"Sender Key ID: {sender_key.thumbprint()}")

Sender Key ID: 46qGI_VLpDqC6ZkT702vOInJHMgRYQeccfiUKHAOCic


#### Build the Message Content

Now, we use the `PCLMessageBuilder` to construct the message.

- We initialize the builder with the sender's and receiver's unique identifiers. According to the PCL exchange data model, these should be persistent identifiers like RORs for organizations.
- `set_content()` populates the main payload of the message. It describes the requested action, which in this case is a measurement. It specifies the target instrument, the sample (using an IGSN), the measurement method, and key parameters for the experiment. This information is structured into a `PCLActionContent` model.
- `add_capability()` allows the sender to specify any technical requirements needed to process the message, such as the ability to perform a specific type of measurement (`xrd.powder.theta-2theta`).

In [15]:
# create a sender node
builder = PCLMessageBuilder(
        sender_id="https://ror.org/03yrm5c26",
        receiver_id="https://ror.org/01bj3aw27"
    )

# set the message content
builder.set_content(
        instrument="urn:aimd:instrument:proto-xrd-01",
        sample="igsn:XYZ12345",
        method="urn:aimd:method:xrd:powder:theta-2theta:v1",
        params={
            "scan_range": {"val": "10 90", "unit": "deg 2theta"},
            "step": {"val": 0.02, "unit": "deg"}
        }
    )

# add the measurement capability
builder.add_capability("xrd.powder.theta-2theta")

#### Sign and Serialize the Message

To guarantee authenticity and integrity, the sender signs the message envelope.

- A `Signer` instance is created using the sender's private key.
- The `builder.sign()` method performs the signing operation. It canonicalizes the envelope data into a consistent string format and generates a Detached JWS. The signature is stored in the `authz` field of the envelope.
- Finally, `builder.build().to_json()` assembles the complete `PCLMessage`, including the RO-Crate metadata, the signed envelope, and the content payload, and serializes it into a JSON-LD string. This string is the \"wire format\" that would be transmitted over a network.

In [20]:
# sign the message
signer = Signer(private_key=sender_key)
builder.sign(signer)

# "send" the message by serializing it to JSON
message_wire_format = builder.build().to_json()
print(f"Message Size: {len(message_wire_format)} bytes")
print(f"Message JSON:\n{json.dumps(json.loads(message_wire_format))}")

Message Size: 2254 bytes
Message JSON:
{"@context": ["https://w3id.org/ro/crate/1.1/context", {"prov": "http://www.w3.org/ns/prov#", "qudt": "http://qudt.org/schema/qudt/", "parameter": "http://schema.org/parameter", "unitText": "http://schema.org/unitText"}], "@graph": [{"@id": "ro-crate-metadata.json", "@type": "CreativeWork", "about": {"@id": "./"}, "conformsTo": {"@id": "https://w3id.org/ro/crate/1.1"}, "identifier": "ro-crate-metadata.json", "name": "RO-Crate Metadata", "text": "Metadata descriptor for PCL Exchange"}, {"@id": "./", "@type": "Dataset", "hasPart": [{"@id": "#envelope"}, {"@id": "#content"}]}, {"@id": "#envelope", "@type": "PCLActionEnvelope", "profile": "https://w3id.org/pcl-profile/action/v1", "schema": "https://w3id.org/pcl-schema/measure-request/v1.0", "identifier": "urn:uuid:6fc2f305-ce1e-4059-890d-428428b979dd", "dateCreated": "2026-01-29T16:10:49.713657Z", "sender": "https://ror.org/03yrm5c26", "receiver": "https://ror.org/01bj3aw27", "action": "request_measur

#### Receive and Verify the Message

This section simulates the actions a receiving node would take upon getting the message. The first and most critical step is to verify the signature to ensure the message is from a trusted sender and has not been altered in transit.

- The `Verifier` is initialized with the sender's *public* key. In a real system, the receiver would need a secure way to retrieve the correct public key associated with the sender's ID.
- We extract the `envelope` dictionary from the received message graph.
- `verifier.verify()` recalculates the signature from the envelope data and compares it to the signature sent in the `authz.jws` field. If they match, the message is authenticated.

In [17]:
# here we would use a transport protocol to send message to receiver node
# for this demo, we just pass the serialized message directly
received_data = json.loads(message_wire_format)

# validate the received message structure
envelope_dict = next(item for item in received_data["@graph"] if item["@id"] == "#envelope")
verifier = Verifier(public_key=sender_key)
is_trusted = verifier.verify(envelope_dict)
    
if is_trusted:
    print("Message successfully authenticated.")
else:
    print("Authentication failed. Signature is invalid.")

Message successfully authenticated.


#### Validate Message Structure

Once the signature is verified, the receiver checks if the message envelope conforms to the expected structure.

- The `validate_structure()` function is called with the envelope dictionary.
- This function uses a JSON Schema (defined in `schemas/envelope.json`) to validate the presence, type, and format of all fields in the envelope. This ensures that the message is well-formed and can be processed reliably.

In [18]:
# validate message structure
valid_struct, err = validate_structure(envelope_dict)
if valid_struct:
    print("JSON schema successfully validated.")
else:
    print(f"JSON schema validation failed:\n{err}")

JSON schema successfully validated.


#### Validate Message Semantics

The final step is to validate the semantic integrity of the entire message graph. While structure validation checks the *form* of the JSON, semantic validation checks the *meaning* and relationships between the different parts of the data.

- The `validate_semantics()` function is called with the full JSON-LD message string.
- It uses SHACL (Shapes Constraint Language) to validate the RDF graph generated from the JSON-LD. The shapes file (`shapes/measurement_request.ttl`) defines rules such as:
    - The `PCLActionContent` node must have an instrument, a sample, and a method.
    - The sample identifier must be a valid IGSN.
- This ensures that the message is not only well-formed but also logically consistent with the PCL data model.

In [19]:
# validate semantics
valid_shapes, err = validate_semantics(message_wire_format, "shapes/measurement_request.ttl")
if valid_shapes:
    print("SHACL shapes successfully validated.")
else:
    print(f"SHACL shapes validation failed:\n{err}")

SHACL shapes successfully validated.
